In [34]:
# Python libarary
import pymysql
import pandas as pd
from bokeh.layouts import gridplot
#from bokeh.plotting import figure, output_notebook, show,Histogram
from bokeh.charts import Histogram,BoxPlot, output_notebook, show
import numpy as np
output_notebook()

Loading BokehJS ...

# Loading Data to My-sql local instance

Creating a new schema flat_iron in a <b mysql(5.7.13) b> local database and uploading the Patient_Diagnosis.csv &
Patient_Treatment.csv 

In [59]:
# Creating a connection 
# note the flat_iron schema was manually created 
conn = pymysql.connect(host='localhost', port=3306, user='root', passwd='', db='flat_iron')
c = conn.cursor()

In [36]:
# creating patient_diagnosis table
create_patient_diagnosis = """
CREATE TABLE if not exists flat_iron.patient_diagnosis (
   Patient_ID INT NOT NULL,
   Diagnosis_Date DATE NOT NULL,
   Diagnosis_Code DECIMAL(7,2) NOT NULL,
   Diagnosis VARCHAR(45) NULL
)
"""
c.execute(create_patient_diagnosis)

# loading the data from Patient_Diagnosis.csv
loading_patient_diagnosis = r"""
LOAD DATA 
INFILE 
'/Users/sroy/Personal/Data_science_experiments/Flat_iron/Patient_Diagnosis.csv'
INTO TABLE  flat_iron.patient_diagnosis
FIELDS TERMINATED BY ',' 
LINES TERMINATED BY '\r\n'
IGNORE 1 ROWS
"""
c.execute(loading_patient_diagnosis)
# commiting
conn.commit()

In [37]:
# creating patient_treatment.csv
create_patient_diagnosis = """
CREATE TABLE if not exists flat_iron.patient_treatment (
   Patient_ID INT NOT NULL,
   Treatment_Date DATE NOT NULL,
   Drug_Code INT NOT NULL
)
"""
c.execute(create_patient_diagnosis)

# loading the data from Patient_Treatment.csv
loading_patient_diagnosis = r"""
LOAD DATA 
INFILE 
'/Users/sroy/Personal/Data_science_experiments/Flat_iron/Patient_Treatment.csv'
INTO TABLE  flat_iron.patient_treatment
FIELDS TERMINATED BY ',' 
LINES TERMINATED BY '\r\n'
IGNORE 1 ROWS
"""
c.execute(loading_patient_diagnosis)
# commiting
conn.commit()

Exercise
1.  How many patients does the clinic have for each cancer diagnosis?

In [38]:
sql_exercise_1 = """
select 
Diagnosis,
count(distinct Patient_ID) as count
from flat_iron.patient_diagnosis
group by 1
order by 2 desc
"""
df_exercise_1 = pd.read_sql(sql_exercise_1,conn)
df_exercise_1

,Diagnosis,count
0,Breast Cancer,26
1,Colon Cancer,12


In [62]:
sql_exercise_2 = """
select 
t1.Patient_ID,
DATEDIFF(min_date,diag_date) as diff
from
(select 
Patient_ID,
min(Diagnosis_Date) as diag_date
from
flat_iron.patient_diagnosis
group by 1 ) t1
inner join

(SELECT Patient_ID,
min(Treatment_Date)  as min_date
from
flat_iron.patient_treatment
group by 1) t2
on t1.Patient_ID = t2.Patient_ID
order by 1 
"""
df_exercise_2 = pd.read_sql(sql_exercise_2,conn)
bins = np.arange(-50,80,10)
hist = Histogram(df_exercise_2, values='diff',
                 title="Time to Start Treatment",bins = list(bins),xlabel= 'Time in days', ylabel = 'count')
show(hist)



In [43]:
df_exercise_2[df_exercise_2['diff'] < 0]

,Patient_ID,diff
11,4374,-47
17,6877,-21


In [48]:
df_exercise_2[df_exercise_2['diff'] > 0].describe()['diff']

count    25.000000
mean     31.760000
std      15.658544
min       2.000000
25%      17.000000
50%      32.000000
75%      43.000000
max      70.000000
Name: diff, dtype: float64

In [26]:
sql_exercise_3 = """
SELECT 
Patient_ID,
Drug_Code,
DATEDIFF(max(Treatment_Date), min(Treatment_Date)) as len_treatment
from
flat_iron.patient_treatment
group by 1,2
order by 2 desc
"""
df_exercise_3 = pd.read_sql(sql_exercise_3,conn)

In [63]:
### Box plot 
p = BoxPlot(df_exercise_3, values='len_treatment', label='Drug_Code',
            title="Boxplot of length of treatment between Drug 201 and Drug 202",color='red')
show(p)

## Print mean of each drug group
print(df_exercise_3.pivot_table( columns= 'Drug_Code' , values= 'len_treatment', aggfunc= np.mean))


### do a Null hypothesis test using permutation test

observed1 = df_exercise_3.pivot_table( columns= 'Drug_Code' , values= 'len_treatment', aggfunc= np.median)[201] 
observed2 = df_exercise_3.pivot_table( columns= 'Drug_Code' , values= 'len_treatment', aggfunc= np.median)[202] 
observed = observed1 - observed2

result = pd.Series(np.zeros(10**4 -1))
def permute_test(d  = 0):
    n = len(df_exercise_3)
    c = len(df_exercise_3[df_exercise_3['Drug_Code']==201])             
    index=np.random.choice(np.arange(0,n,1), size = c, replace = False)
    not_index = np.setdiff1d(np.arange(0,n,1),index , assume_unique = True)
    return df_exercise_3['len_treatment'].iloc[index].median() - df_exercise_3['len_treatment'].iloc[not_index].median()
result=result.apply(permute_test)
# calculating P value
# calculating P value
p_value = ((result >= observed).sum() +1)/(10**4)
#### Print p value
print('p value is : ')
print(p_value)

Drug_Code
201    72.000000
202    10.769231
Name: len_treatment, dtype: float64
p value is : 
0.0001


In [56]:
conn.close()


array([-50, -48, -46, -44, -42, -40, -38, -36, -34, -32, -30, -28, -26,
       -24, -22, -20, -18, -16, -14, -12, -10,  -8,  -6,  -4,  -2,   0,
         2,   4,   6,   8,  10,  12,  14,  16,  18,  20,  22,  24,  26,
        28,  30,  32,  34,  36,  38,  40,  42,  44,  46,  48,  50,  52,
        54,  56,  58,  60,  62,  64,  66,  68,  70,  72,  74,  76,  78])